In [ ]:
# Reddit 데이터셋 불러오기
from torch_geometric.datasets import Reddit

dataset = Reddit(root='./data/Reddit')
data = dataset[0]  # 데이터셋의 첫 번째 그래프 불러오기

In [ ]:
train_nodes = set([i for i, mask in enumerate(train_mask) if mask])
test_nodes = set([i for i, mask in enumerate(test_mask) if mask])

# 본 적 없는 노드가 있는지 확인
unseen_nodes = test_nodes - train_nodes

if unseen_nodes:
    print(f"There are {len(unseen_nodes)} nodes in the test set that are not seen in the training set.")
else:
    print("All nodes in the test set are seen in the training set.")
